In [1]:
!pip install awswrangler
!pip install imblearn

In [9]:
mybucket = 'dlocal-eu1-office-live-sagemaker-mharley/dev'

from rule_finder import *


In [10]:
import awswrangler as wr
import boto3
import pandas as pd
import numpy as np
import random
import itertools
import re
import datetime

In [11]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('precision', 2)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.options.display.max_colwidth = 200

In [12]:
# Parameters for the query extraction

min_date = '2023-03-01'
max_date = '2023-05-01'
mid_analisis = ['50769']   
country_analisis = ['ALL']
card_type_analisis = ['DEBIT','CREDIT']
brand_analisis = ['ALL']
bin_analisis = ['ALL']
model_analisis = ['mx','retailV2']

In [13]:


velocities_and_morphings = [
    'cnt_same_card_app_7d',
    'sum_amt_same_card_app_7d',
    'cnt_same_card_app_30d',
    'sum_amt_same_card_app_30d',
    
    'cnt_same_address_app_7d',
    'sum_amt_same_address_app_7d',
    'cnt_same_address_app_30d',
    'sum_amt_same_address_app_30d',
    
    'cnt_same_device_id_app_7d',
    'sum_amt_same_device_id_app_7d',
    'cnt_same_device_id_app_30d',
    'sum_amt_same_device_id_app_30d',

    
    'cnt_same_card_diff_address_app_1d',
    'cnt_same_card_diff_address_app_7d',
    'cnt_same_card_diff_address_app_30d',
    'cnt_same_card_diff_address_app_90d',
    
    'cnt_same_card_diff_address_1d',
    'cnt_same_card_diff_address_7d',
    'cnt_same_card_diff_address_30d',
    'cnt_same_card_diff_address_90d',
    
    'cnt_same_address_diff_card_1d',
    'cnt_same_address_diff_card_30d',
    'cnt_same_address_diff_card_90d',
    'cnt_same_address_diff_card_7d',
    
    'cnt_same_card_diff_email_app_1d',
    'cnt_same_card_diff_email_app_7d',
    'cnt_same_card_diff_email_app_30d',
    'cnt_same_card_diff_email_app_90d',
    
    'cnt_same_card_diff_email_1d',
    'cnt_same_card_diff_email_7d',
    'cnt_same_card_diff_email_30d',
    'cnt_same_card_diff_email_90d',

    'cnt_same_email_diff_card_1d',
    'cnt_same_email_diff_card_30d',
    'cnt_same_email_diff_card_90d',
    'cnt_same_email_diff_card_7d',

    'cnt_same_email_diff_card_app_1d',
    'cnt_same_email_diff_card_app_30d',
    'cnt_same_email_diff_card_app_90d',
    'cnt_same_email_diff_card_app_7d'

    
    
    ]


In [14]:
%%time
df = create_df(min_date, max_date, mid_analisis, country_analisis, card_type_analisis, brand_analisis, bin_analisis, model_analisis, velocities_and_morphings)


Completitud del modelo mx = 99%
Completitud del modelo retailV2 = 100%
Cantidad de pagos = 263262

CPU times: user 24.2 s, sys: 171 ms, total: 24.3 s
Wall time: 2min 39s


In [94]:
df.head()

,case_id,date_created_date,year_month,bin,usd_amount,rule_type,score_mx,score_retailV2,cnt_same_card_app_7d,sum_amt_same_card_app_7d,cnt_same_card_app_30d,sum_amt_same_card_app_30d,cnt_same_address_app_7d,sum_amt_same_address_app_7d,cnt_same_address_app_30d,sum_amt_same_address_app_30d,cnt_same_device_id_app_7d,sum_amt_same_device_id_app_7d,cnt_same_device_id_app_30d,sum_amt_same_device_id_app_30d,cnt_same_card_diff_address_app_1d,cnt_same_card_diff_address_app_7d,cnt_same_card_diff_address_app_30d,cnt_same_card_diff_address_app_90d,cnt_same_card_diff_address_1d,cnt_same_card_diff_address_7d,cnt_same_card_diff_address_30d,cnt_same_card_diff_address_90d,cnt_same_address_diff_card_1d,cnt_same_address_diff_card_30d,cnt_same_address_diff_card_90d,cnt_same_address_diff_card_7d,cnt_same_card_diff_email_app_1d,cnt_same_card_diff_email_app_7d,cnt_same_card_diff_email_app_30d,cnt_same_card_diff_email_app_90d,cnt_same_card_diff_email_1d,cnt_same_card_diff_email_7d,cnt_same_card_diff_email_30d,cnt_same_card_diff_email_90d,cnt_same_email_diff_card_1d,cnt_same_email_diff_card_30d,cnt_same_email_diff_card_90d,cnt_same_email_diff_card_7d,cnt_same_email_diff_card_app_1d,cnt_same_email_diff_card_app_30d,cnt_same_email_diff_card_app_90d,cnt_same_email_diff_card_app_7d,incoming_qty,flag_app_pf,flag_app_final,cbk_flag,incoming_amt,app_pf_amt,app_final_amt,cbk_amt
0,3502904035,2023-03-15,2023_3,481515,59.04,0,678,1,0,0,1,60,0,0,0,0,0,0,7,464,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,59.04,59.04,59.04,0.00
1,3936304597,2023-04-01,2023_4,547068,60.80,0,226,1,0,0,1,62,0,0,0,0,0,0,1,62,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,60.80,60.80,60.80,0.00
2,3951833821,2023-04-04,2023_4,481514,60.71,0,109,1,0,0,1,62,0,0,0,0,48,5326,120,12209,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,60.71,60.71,60.71,0.00
3,3454697816,2023-03-05,2023_3,402766,31.23,0,929,11,0,0,3,580,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,3,3,0,0,1,0,0,0,0,0,1,1,1,0,31.23,31.23,31.23,0.00
4,3459510926,2023-03-06,2023_3,481515,62.46,0,566,0,0,0,0,0,0,0,0,0,2,135,6,393,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,62.46,62.46,62.46,0.00


In [95]:
app = df[df['flag_app_final']==1]
app.groupby('year_month')['usd_amount'].sum()


year_month
2023_3   6,204,171.81
2023_4   6,904,512.95
2023_5     230,403.79
Name: usd_amount, dtype: float64

In [96]:
df['year_month'].value_counts()

2023_3    130273
2023_4    128835
2023_5      4154
Name: year_month, dtype: Int64

In [101]:
df.groupby('year_month')['usd_amount'].sum()


year_month
2023_3   14,572,042.99
2023_4   12,475,364.37
2023_5      415,904.22
Name: usd_amount, dtype: float64

In [102]:
df.groupby('year_month')['cbk_flag'].sum()

year_month
2023_3    303
2023_4    585
2023_5     18
Name: cbk_flag, dtype: Int32

In [103]:
df.groupby('year_month')['usd_amount'].sum()


year_month
2023_3   14,572,042.99
2023_4   12,475,364.37
2023_5      415,904.22
Name: usd_amount, dtype: float64

In [100]:
cbk.groupby('year_month')['usd_amount'].sum()


year_month
2023_3    67,215.72
2023_4   140,270.29
2023_5     2,456.96
Name: usd_amount, dtype: float64

In [36]:
df['flag_hardware'] = np.where((df['usd_amount']<=480) & (df['usd_amount']>=460) ,1,0)

In [45]:
df['flag_hardware'].value_counts(normalize=True)

0   0.90
1   0.10
Name: flag_hardware, dtype: float64

In [25]:
# Se guarda un respaldo del df (para no correr query de vuelta para mismo análisis)
 df.reset_index().to_feather('test.starlink')

In [21]:
# Activar en caso de querer trabajar a partir del df guardado
 df = pd.read_feather('test.glup1_12')

In [62]:
cbk = df[df['cbk_flag']== 1]

In [42]:
df['cbk_flag'].value_counts()

0    28085
1      190
Name: cbk_flag, dtype: Int64

In [43]:
cbk['year_month'].value_counts()

2023_3    146
2023_2     44
Name: year_month, dtype: Int64

In [44]:
cbk['flag_hardware'].value_counts()

1    95
0    95
Name: flag_hardware, dtype: int64

In [13]:
df.head()

,case_id,date_created_date,year_month,bin,usd_amount,rule_type,score_mx,score_retailV2,cnt_same_card_app_7d,sum_amt_same_card_app_7d,cnt_same_card_app_30d,sum_amt_same_card_app_30d,cnt_same_address_app_7d,sum_amt_same_address_app_7d,cnt_same_address_app_30d,sum_amt_same_address_app_30d,cnt_same_device_id_app_7d,sum_amt_same_device_id_app_7d,cnt_same_device_id_app_30d,sum_amt_same_device_id_app_30d,cnt_same_card_diff_address_app_1d,cnt_same_card_diff_address_app_7d,cnt_same_card_diff_address_app_30d,cnt_same_card_diff_address_app_90d,cnt_same_card_diff_address_1d,cnt_same_card_diff_address_7d,cnt_same_card_diff_address_30d,cnt_same_card_diff_address_90d,cnt_same_address_diff_card_1d,cnt_same_address_diff_card_30d,cnt_same_address_diff_card_90d,cnt_same_address_diff_card_7d,cnt_same_card_diff_email_app_1d,cnt_same_card_diff_email_app_7d,cnt_same_card_diff_email_app_30d,cnt_same_card_diff_email_app_90d,cnt_same_card_diff_email_1d,cnt_same_card_diff_email_7d,cnt_same_card_diff_email_30d,cnt_same_card_diff_email_90d,cnt_same_email_diff_card_1d,cnt_same_email_diff_card_30d,cnt_same_email_diff_card_90d,cnt_same_email_diff_card_7d,cnt_same_email_diff_card_app_1d,cnt_same_email_diff_card_app_30d,cnt_same_email_diff_card_app_90d,cnt_same_email_diff_card_app_7d,incoming_qty,flag_app_pf,flag_app_final,cbk_flag,incoming_amt,app_pf_amt,app_final_amt,cbk_amt
0,3521978975,2023-03-19,2023_3,546616,479.69,1,973,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,0,0,0,0,1,0,0,0,479.69,0.00,0.00,0.00
1,3303860856,2023-01-31,2023_1,409355,45.41,1,-1,1,0,0,1,471,0,0,0,0,128,29392,200,34221,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,0,0,0,0,0,1,1,1,0,45.41,45.41,45.41,0.00
2,3536283396,2023-03-22,2023_3,549033,0.02,1,730,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,10,6,0,0,0,0,1,1,0,0,0.02,0.02,0.00,0.00
3,3489305251,2023-03-12,2023_3,524886,55.52,0,169,1,0,0,1,55,0,0,0,0,0,0,3,184,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,55.52,55.52,55.52,0.00
4,3973259032,2023-04-08,2023_4,411054,46.27,0,702,1,0,0,1,8,0,0,0,0,54,8529,167,25288,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,46.27,46.27,46.27,0.00


In [46]:
green_bins, yellow_bins = bin_segmentation (df)


green_bins = []
yellow bins = ['530691']


In [47]:
%%time
df_reducido, combinations_dict = create_combinations (df, green_bins, yellow_bins, model_analisis)



Rows in the dataframe:  23354

Total combinations: 
159936

Features incluidos: 
score_mx                     0.14
usd_amount                   0.10
score_retailV2               0.06
rule_type                    0.04
bin                          0.03
cnt_same_device_id_app_30d   0.03
cnt_same_device_id_app_7d    0.02
Name: cbk_flag, dtype: float64

Features descartados:  ['cnt_same_card_app_7d', 'sum_amt_same_card_app_7d', 'cnt_same_card_app_30d', 'sum_amt_same_card_app_30d', 'cnt_same_address_app_7d', 'sum_amt_same_address_app_7d', 'cnt_same_address_app_30d', 'sum_amt_same_address_app_30d', 'sum_amt_same_device_id_app_7d', 'sum_amt_same_device_id_app_30d', 'cnt_same_card_diff_address_app_1d', 'cnt_same_card_diff_address_app_7d', 'cnt_same_card_diff_address_app_30d', 'cnt_same_card_diff_address_app_90d', 'cnt_same_card_diff_address_1d', 'cnt_same_card_diff_address_7d', 'cnt_same_card_diff_address_30d', 'cnt_same_card_diff_address_90d', 'cnt_same_address_diff_card_1d', 'cnt_same_addre

In [48]:
%%time
# year_month = ['2022_11','2022_12']
# previous_rules = [123, 183]

it1 = Itaration(combinations_dict, df_reducido, limit=1000, previous_rules = [], year_month = 'ALL' )

Reminder:
rule_type = 0 --> Positive_user
rule_type = 1 --> Default
bin = 0 --> Green bin
bin = 1 --> Yellow bin
bin = 2 --> Others bins


CPU times: user 564 ms, sys: 3.8 ms, total: 568 ms
Wall time: 567 ms


In [49]:
pd.set_option('display.max_colwidth', 1000)
pd.options.display.float_format = '{:.2f}'.format

In [50]:
it1[
    (it1['Extra recall amt'] > 30)
    & (it1['Extra recall qty'] > 20)].sort_values('Precision amt', ascending=False
        ).head(10)




,Condition,Delta app_Dlocal qty,New App Dlocal qty,Delta app_final qty,New App final qty,Delta cbk qty,New cbk qty,Extra recall qty,Precision qty,//,Delta app_Dlocal amt,New App Dlocal amt,Delta app_final amt,New App final amt,Delta cbk amt,New cbk amt,Extra recall amt,Precision amt
id_rule,,,,,,,,,,,,,,,,,,
24689,(arrit['score_mx']>=785) & (arrit['usd_amount']>=474.78) & (arrit['score_retailV2']>=8) & (arrit['rule_type']>=1) & (arrit['bin']>=3),-0.48,79.65,-0.27,30.13,-0.56,1.65,25.79,62.82,//,-1.84,78.92,-1.04,56.65,-1.09,2.47,31.98,63.29
27083,(arrit['score_mx']>=811) & (arrit['usd_amount']>=467.92) & (arrit['score_retailV2']>=6) & (arrit['rule_type']>=1),-0.61,79.52,-0.36,30.05,-0.68,1.53,31.58,58.82,//,-2.31,78.92,-1.36,56.33,-1.34,2.22,39.08,59.15
30335,(arrit['score_mx']>=833) & (arrit['usd_amount']>=467.92) & (arrit['score_retailV2']>=6),-0.67,79.46,-0.38,30.02,-0.70,1.51,32.63,56.88,//,-2.58,78.92,-1.48,56.21,-1.38,2.18,40.38,56.23
101339,(arrit['score_mx']>=954) & (arrit['score_retailV2']>=6) & (arrit['rule_type']>=1),-0.92,79.21,-0.52,29.89,-0.66,1.55,31.05,40.14,//,-2.46,78.92,-1.41,56.28,-1.08,2.48,31.96,46.56
11603,(arrit['score_mx']>=622) & (arrit['usd_amount']>=474.78) & (arrit['score_retailV2']>=7) & (arrit['rule_type']>=1),-0.76,79.37,-0.42,29.98,-0.63,1.58,29.47,46.28,//,-2.91,78.92,-1.62,56.07,-1.23,2.33,36.53,46.35
43349,(arrit['score_mx']>=863) & (arrit['usd_amount']>=467.92) & (arrit['score_retailV2']>=4) & (arrit['bin']>=3),-0.75,79.38,-0.46,29.95,-0.67,1.54,31.58,45.80,//,-2.89,78.92,-1.78,55.91,-1.32,2.24,39.10,45.26
101315,(arrit['score_mx']>=954) & (arrit['score_retailV2']>=5) & (arrit['rule_type']>=1),-1.06,79.07,-0.62,29.79,-0.74,1.47,34.74,37.71,//,-2.81,78.92,-1.64,56.05,-1.19,2.37,35.28,44.21
68741,(arrit['score_mx']>=927) & (arrit['score_retailV2']>=8) & (arrit['bin']>=3),-1.28,78.85,-0.71,29.70,-0.88,1.33,41.05,38.81,//,-3.53,78.92,-2.03,55.66,-1.48,2.08,43.66,44.18
36227,(arrit['score_mx']>=847) & (arrit['score_retailV2']>=15) & (arrit['rule_type']>=1),-1.03,79.11,-0.58,29.82,-0.75,1.46,35.26,40.36,//,-2.87,78.92,-1.71,55.98,-1.23,2.33,36.66,44.17


In [63]:
%%time
# year_month = ['2022_11','2022_12']
# previous_rules = [123, 183]

it2 = Itaration(combinations_dict, df_reducido, limit=5000, previous_rules = [12564], year_month = 'ALL' )

Reminder:
rule_type = 0 --> Positive_user
rule_type = 1 --> Default
bin = 0 --> Green bin
bin = 1 --> Yellow bin
bin = 2 --> Others bins


CPU times: user 1min 31s, sys: 117 ms, total: 1min 31s
Wall time: 1min 31s


In [64]:
pd.set_option('display.max_colwidth', 1000)
pd.options.display.float_format = '{:.2f}'.format

In [65]:
it2[
    (it2['Extra recall amt'] > 10)
    & (it2['Extra recall qty'] > 10)].sort_values('Precision amt', ascending=False
        ).head(30)

,Condition,Delta app_Dlocal qty,New App Dlocal qty,Delta app_final qty,New App final qty,Delta cbk qty,New cbk qty,Extra recall qty,Precision qty,//,Delta app_Dlocal amt,New App Dlocal amt,Delta app_final amt,New App final amt,Delta cbk amt,New cbk amt,Extra recall amt,Precision amt
id_rule,,,,,,,,,,,,,,,,,,
3920,(arrit['score_mx']>=224) & (arrit['usd_amount']>=21.25) & (arrit['rule_type']>=1) & (arrit['score_retailV2']>=94),-0.62,61.58,-0.36,51.93,-0.25,2.10,11.16,37.81,//,-1.22,57.78,-0.68,46.98,-0.52,2.74,17.17,39.23
90128,(arrit['score_mx']>=962) & (arrit['usd_amount']>=22.78) & (arrit['rule_type']>=1) & (arrit['score_retailV2']>=6),-0.57,61.62,-0.34,51.96,-0.22,2.13,10.15,37.04,//,-1.13,57.78,-0.64,47.02,-0.49,2.77,15.94,38.81
47351,(arrit['score_mx']>=906) & (arrit['usd_amount']>=22.78) & (arrit['rule_type']>=1) & (arrit['score_retailV2']>=57),-0.58,61.62,-0.34,51.96,-0.22,2.13,10.17,37.22,//,-1.16,57.78,-0.65,47.01,-0.49,2.77,16.09,38.69
29285,(arrit['score_mx']>=843) & (arrit['usd_amount']>=22.78) & (arrit['bin']>=3) & (arrit['rule_type']>=1) & (arrit['score_retailV2']>=44),-0.70,61.50,-0.41,51.88,-0.27,2.08,12.40,36.89,//,-1.39,57.77,-0.79,46.87,-0.60,2.66,19.56,38.49
65693,(arrit['score_mx']>=936) & (arrit['usd_amount']>=22.78) & (arrit['rule_type']>=1) & (arrit['score_retailV2']>=36),-0.60,61.60,-0.35,51.94,-0.23,2.12,10.45,36.92,//,-1.19,57.78,-0.66,47.00,-0.50,2.76,16.33,38.37
40940,(arrit['score_mx']>=890) & (arrit['usd_amount']>=21.25) & (arrit['bin']>=3) & (arrit['rule_type']>=1) & (arrit['score_retailV2']>=80),-0.58,61.61,-0.35,51.95,-0.23,2.12,10.34,36.72,//,-1.14,57.78,-0.64,47.02,-0.48,2.78,15.79,38.25
71513,(arrit['score_mx']>=944) & (arrit['usd_amount']>=21.25) & (arrit['bin']>=3) & (arrit['rule_type']>=1) & (arrit['score_retailV2']>=44),-0.65,61.54,-0.39,51.90,-0.26,2.09,11.74,36.84,//,-1.23,57.78,-0.70,46.95,-0.53,2.73,17.34,38.23
32354,(arrit['score_mx']>=857) & (arrit['usd_amount']>=22.78) & (arrit['bin']>=3) & (arrit['rule_type']>=1) & (arrit['score_retailV2']>=53),-0.64,61.56,-0.37,51.92,-0.25,2.10,11.18,36.86,//,-1.28,57.78,-0.71,46.94,-0.54,2.72,17.57,38.21
84014,(arrit['score_mx']>=956) & (arrit['usd_amount']>=22.78) & (arrit['rule_type']>=1) & (arrit['score_retailV2']>=9),-0.61,61.59,-0.35,51.94,-0.23,2.12,10.60,36.69,//,-1.19,57.78,-0.67,46.99,-0.50,2.76,16.44,38.20


In [67]:
show_rules (combinations_dict, df_reducido, [12564,3920])

,Condition,Delta app_Dlocal qty,New App Dlocal qty,Delta app_final qty,New App final qty,Delta cbk qty,New cbk qty,Extra recall qty,Precision qty,//,Delta app_Dlocal amt,New App Dlocal amt,Delta app_final amt,New App final amt,Delta cbk amt,New cbk amt,Extra recall amt,Precision amt
id_rule,,,,,,,,,,,,,,,,,,
12564,(arrit['score_mx']>=701) & (arrit['bin']>=3) & (arrit['score_retailV2']>=9) & (arrit['cnt_same_card_diff_address_7d']>=1),-3.06,59.14,-1.75,50.54,-0.87,1.48,39.15,27.55,//,-3.89,57.63,-2.09,45.56,-1.34,1.92,43.62,32.33
3920,(arrit['score_mx']>=224) & (arrit['usd_amount']>=21.25) & (arrit['rule_type']>=1) & (arrit['score_retailV2']>=94),-0.62,61.58,-0.36,51.93,-0.25,2.10,11.16,37.81,//,-1.22,57.78,-0.68,46.98,-0.52,2.74,17.17,39.23


In [72]:
df['usd_amount'].describe()

count   346925.00
mean        16.54
std          7.67
min          1.02
25%         12.68
50%         15.04
75%         17.57
max         82.68
Name: usd_amount, dtype: float64

In [69]:
df['score_mx'].describe()

count   346925.00
mean       658.14
std        308.52
min         -1.00
25%        400.00
50%        759.00
75%        939.00
max       1000.00
Name: score_mx, dtype: float64

In [71]:
df['score_retailV2'].describe()

count   346925.00
mean        63.99
std        128.40
min         -1.00
25%          3.00
50%         11.00
75%         58.00
max        997.00
Name: score_retailV2, dtype: float64